## Module Import 

In [ ]:
import sys
import math
import numpy as np
import matplotlib.pyplot as plt
import h5py
import re
import os
import time as t
import pandas as pd

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.python.keras.layers import Dense
np.set_printoptions(precision=8)
seed=98

print(tf. __version__)

2.7.0


In [ ]:
# change the directory to data location
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/PINN')

Mounted at /content/drive


In [ ]:
# Read the file
file = 'major_chemical_marker.h5'
f = h5py.File(file, 'r')

In [ ]:
# Get the feature and the label
# The dataset have larger points HHR region [data= Augumentation]
dataset = np.array(f.get('major_chemical_marker'))
print(dataset.shape)

(51966851, 5)


In [ ]:
# Separate the feature and the label
X_old = dataset[:,:-1]
nh = X_old[:,0:1]
oh = X_old[:,2:3]
X = np.c_[nh,oh]
Y = dataset[:,[-1]]
#print(X,Y.shape)

In [ ]:
#shuffle the dataset
X,Y = shuffle(X, Y, random_state=seed)
print(X, Y)

[[2.81994056e-04 7.90773076e-04]
 [5.49053948e-05 2.29734957e-04]
 [2.23566258e-05 8.72315094e-03]
 ...
 [3.13814708e-05 9.34661739e-03]
 [1.40960456e-05 5.31098386e-03]
 [4.33951609e-05 1.05819294e-04]] [[5.4436895e+09]
 [1.4833097e+09]
 [1.4732527e+09]
 ...
 [1.5068352e+09]
 [9.4112301e+08]
 [1.7772887e+09]]


In [ ]:
# create the normalization fit 
scale_X = MinMaxScaler()
scale_Y = MinMaxScaler()

# The fit function
print(scale_X.fit(X))
print(scale_Y.fit(Y))

MinMaxScaler()
MinMaxScaler()


In [ ]:
#test_train split 
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1)
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(46770165, 2) (5196686, 2) (46770165, 1) (5196686, 1)


In [ ]:
# normalize the train data and label
X_train_n = scale_X.transform(X_train)
Y_train_n = scale_Y.transform(Y_train)
print(X_train_n)
print(Y_train_n)

[[0.3804782  0.4289206 ]
 [0.5737346  0.2145094 ]
 [0.21089937 0.39399308]
 ...
 [0.47351608 0.07820035]
 [0.01867825 0.06101279]
 [0.03219866 0.00700167]]
[[0.22912833]
 [0.61096185]
 [0.2700133 ]
 ...
 [0.32622337]
 [0.00186322]
 [0.0327349 ]]


In [ ]:
# Create the chunk 
X_chunk = tf.Variable(X_train_n[0:5000000,:])
Y_chunk = tf.Variable(Y_train_n[0:5000000,:])
print(X_chunk, Y_chunk)

<tf.Variable 'Variable:0' shape=(5000000, 2) dtype=float32, numpy=
array([[0.3804782 , 0.4289206 ],
       [0.5737346 , 0.2145094 ],
       [0.21089937, 0.39399308],
       ...,
       [0.04799581, 0.00473209],
       [0.02664264, 0.04414031],
       [0.04581539, 0.08583519]], dtype=float32)> <tf.Variable 'Variable:0' shape=(5000000, 1) dtype=float32, numpy=
array([[0.22912833],
       [0.61096185],
       [0.2700133 ],
       ...,
       [0.08711284],
       [0.02786726],
       [0.00736997]], dtype=float32)>


In [ ]:
# Creaate the batch slices.
train_data = tf.data.Dataset.from_tensor_slices((X_chunk, Y_chunk))
train_data = train_data.batch(1024, drop_remainder=True)
print(train_data)
# test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
# test_data = test_data.batch(1024, drop_remainder=True)

<BatchDataset shapes: ((1024, 2), (1024, 1)), types: (tf.float32, tf.float32)>


In [ ]:
# Get the pretrained model
# Read the file
file = 'nh_oh.h5'
model = tf.keras.models.load_model(file)

# Get summary
model.summary()


Model: "Heat_release_rate"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 h_layer1 (Dense)            (None, 40)                120       
                                                                 
 h_layer2 (Dense)            (None, 24)                984       
                                                                 
 h_layer3 (Dense)            (None, 24)                600       
                                                                 
 output_layers (Dense)       (None, 1)                 25        
                                                                 
Total params: 1,729
Trainable params: 1,729
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# The Ki the is respective exponents of the species massfraction and temperature, which could yeild us a perfect reconstruction of the HRR
#k1, k2, k3, k4 = tf.Variable([1.0], dtype=tf.float32),tf.Variable([-0.5], dtype=tf.float32),tf.Variable([-0.75], dtype=tf.float32),tf.Variable([1.0], dtype=tf.float32)
k1,k2,k3 = tf.Variable(2.0),tf.Variable(-2.0),tf.Variable(2.0)

In [ ]:
# The customized mean square error, taking the batch data at any instance
def customized_MSE(true, pred):
    MSE = tf.reduce_mean(tf.square(true-pred))
    return MSE
# The customized loss taking the non-linear equation as residual
def total_mse(MSE_NN, X, Y, k1, k2, k3):# the MSE_NN is the mean squared error of the neural network 
    residual = tf.square(tf.math.log(Y) - (k1*tf.math.log(X[:,0:1]) + k2* tf.math.log(X[:,1:2]) + tf.math.log(k3)))
    MSE_func = tf.reduce_mean(residual)
    total =  MSE_NN + MSE_func
    return total

In [ ]:
# Optimizer 
# optimizers = tf.keras.optimizers.SGD(learning_rate=0.03,momentum=0.8)
optimizers = tf.keras.optimizers.Adam(
    learning_rate=0.009, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')

In [ ]:
#Training with customized optimization technic (Automatic-Gradient)
epochs = 100
# lr_rate = 0.07


for epoch in range(epochs):
  print("Starting a new epoch",epoch+1)
  for step, (X_batch, Y_batch) in enumerate(train_data):
    with tf.GradientTape(persistent=True) as tape:
      Y_pred = model(X_batch)
      current_loss = customized_MSE(Y_batch,Y_pred)
      new_loss = total_mse(current_loss, X_batch, Y_batch, k1, k2, k3)

    # The gradient check  
    gradient = tape.gradient(new_loss, [k1,k2,k3])
    grad = tape.gradient(current_loss, model.trainable_variables)

    #Assign new variables to the model using optimizer isntead of sub assign
    optimizers.apply_gradients(zip(grad, model.trainable_variables))
    optimizers.apply_gradients(zip(gradient, [k1,k2,k3]))

    #Update the Ki to reduce the residual
    # k1.assign_sub(gradient[0]*lr_rate)
    # k2.assign_sub(gradient[1]*lr_rate)
    # k3.assign_sub(gradient[2]*lr_rate)
  
  print("k1:{} k2:{} k3:{} ".format(k1.numpy(),k2.numpy(),k3.numpy()))
  print("current_Loss: {} new_loss: {}".format(current_loss.numpy(), new_loss.numpy()))  

Starting a new epoch 1
k1:0.8753795623779297 k2:-0.04514443874359131 k3:0.5742367506027222 
current_Loss: 0.004508631303906441 new_loss: 0.3816887438297272
Starting a new epoch 2
k1:0.8684256076812744 k2:-0.049870815128088 k3:0.5817666053771973 
current_Loss: 0.004463497083634138 new_loss: 0.38063159584999084
Starting a new epoch 3
k1:0.8683488368988037 k2:-0.04997862130403519 k3:0.5818014740943909 
current_Loss: 0.004423074424266815 new_loss: 0.3806000351905823
Starting a new epoch 4
k1:0.8683484792709351 k2:-0.049979131668806076 k3:0.5818014740943909 
current_Loss: 0.004449429921805859 new_loss: 0.3806264102458954
Starting a new epoch 5
k1:0.8683484792709351 k2:-0.04997912421822548 k3:0.5818014740943909 
current_Loss: 0.00448231864720583 new_loss: 0.3806593120098114
Starting a new epoch 6
k1:0.8683484792709351 k2:-0.049979131668806076 k3:0.5818015336990356 
current_Loss: 0.004468971863389015 new_loss: 0.3806459903717041
Starting a new epoch 7
k1:0.8683484196662903 k2:-0.0499791204929